In [1]:
from pytezos import pytezos

from config import KEY, SHELL_URI, JUSTER_ADDRESS
client = pytezos.using(key=KEY, shell=SHELL_URI)
contract = client.contract(JUSTER_ADDRESS)

In [2]:
from run_maker import generate_event_lines
generate_event_lines()

generated 27 event lines


In [3]:
from dipdup import JusterDipDupClient
from executors import WithdrawCaller
import time
from utility import timestamp_to_date

dd_client = JusterDipDupClient()
REWARD_SPLIT_FEE_AFTER = 24*60*60
closed_before = timestamp_to_date(time.time() - REWARD_SPLIT_FEE_AFTER)
withdrawable_events = dd_client.query_withdrawable_events(closed_before)

In [4]:
# withdrawable_events

In [5]:
contract.storage['events'][7]()

KeyError: 7

### Why 7 and 12 events have unwithdrawn positions?

In [12]:
import requests
import json
import pandas as pd
from urllib.parse import urlencode

method = f'https://api.florencenet.tzkt.io/v1/accounts/{JUSTER_ADDRESS}/operations?'
params = urlencode({
    'limit': 1000,
    'parameter.eventId.eq': 2428,
    # 'parameters': 'withdraw'
})
response = requests.get(method + params)

data = json.loads(response.text)
df = pd.json_normalize(data)
len(df)

3

In [15]:
[d['parameter'] for d in data if 'parameter' in d]

[{'entrypoint': 'provideLiquidity',
  'value': {'eventId': '2428',
   'maxSlippage': '1000000',
   'expectedRatioBelow': '1',
   'expectedRatioAboveEq': '1'}}]

In [8]:
withdrawals = df[ df['parameter.entrypoint'] == 'withdraw' ]
withdrawals

KeyError: 'parameter.entrypoint'

In [9]:
withdrawals.hash.to_dict()

NameError: name 'withdrawals' is not defined

In [10]:
withdrawals.iloc[0]['hash'].to_dict()

NameError: name 'withdrawals' is not defined

In [11]:
# https://better-call.dev/florencenet/opg/opF9uh7hbgd7wzmp7HYQ6vwVvrYaQwFhB9syyeCgnkeU15B8jiG/contents
# participants 2 -> 1

w12.iloc[0].hash

NameError: name 'w12' is not defined

### Juster-Maker:
Tool allowing to run events in cycle

### Создание линеек ивентов:
- Линейка ивентов - это набор из однотипных ивентов которые создаются друг за другом с наложением периода принятия ставок и периода измерения.
- Тип ивента может быть определён по 3 параметрам:
    - валютная пара
    - target dynamics
    - period
- Также ивенты могут отличаются liquidity_percent но сейчас я считаю что liquidity percent привязыватеся к period + target_dynamics

- TODO: вынести в отдельный ipynb или отдельный скрипт (если отдельный скрипт - хорошо было бы описать этот процесс в readme.MD, либо может нужно вообще CLI утилиту сделать в которой одна из точек входа это генерация линеек)

### TODO: remake all using apscheduler
    https://apscheduler.readthedocs.io/en/stable/

### TODO: Autowithdrawal after 24 hours
    - for all events check if it is closed for more than 24 hours, if it is, make withdrawal transactions for all participants

### TODO: add here pusher & startMeasurement/close?
### TODO: in event creation somehow check that event successfully created and if it is not: make another in 5 minutes?

### TODO: it is better not to return transactions in pool by BulkSigner, but reemmit all transactions by EventEmmitter / LiquidityProvider instead
- so EventEmmitter can just check either transaction was successfull or not (after some pause for 10 mins for example) and only add period to timer if it was succeed
- the same with LiquidityProvider who just checks only for the last possible events in line, so if it is new event: he would use it

### ++ TODO: PROVIDE LIQUIDITY for this events

### ++ TODO: OK, в общем проще всего мне кажется вот так:
    - для каждой линейки ивентов создаётся свой event_emitter, он проверяет что пришло время запустить ивет сравнивая текущее значение таймера с таргетом. И после запуска ивента он переставляет таймер дальше.
    - с определённой частотой (например раз в 20 секунд) таймер проверяется и если время больше чем в таймере - транза с новым ивентом закидывается в очередь и подхватывается BulkSender

### TODO: maybe make next_event func/iterator that returns time and params for the next events

### TODO: need to have some tests for LoopExecutor:
- for example that it runned function K times in given time

### TODO: need to test all this code before running in florencenet

### TODO: provide liquidity
### TODO: pushing prices